In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import numpy as np
import pandas as pd

import kraft

In [ ]:
output_directory_path = "../output/"

In [ ]:
feature_name = "protein"

feature_x_cell_line = pd.read_csv(
    "/media/kwat/CarrotCake/garden/data/cancer_cell_line/{}/gene_x_cell_line.tsv".format(
        feature_name
    ),
    sep="\t",
    index_col=0,
)

# feature_x_cell_line

In [ ]:
sea_method = "classic"

gene_set_x_cell_line = pd.read_csv(
    "/media/kwat/CarrotCake/garden/data/cancer_cell_line/gene_set/gene_set_x_sample_x_{}.tsv".format(
        sea_method
    ),
    sep="\t",
    index_col=0,
)

# gene_set_x_cell_line

In [ ]:
feature_x_cell_line.columns & gene_set_x_cell_line.columns

In [ ]:
gene_sets = gene_set_x_cell_line.index.to_numpy()

feature_to_gene_sets = {}

feature_to_n_gene_set = {}

for feature in feature_x_cell_line.index.to_numpy():

    _feature_ = "_{}_".format(feature)

    gene_sets_matched = tuple(
        gene_set for gene_set in gene_sets if _feature_ in gene_set
    )

    if 0 < len(gene_sets_matched):

        feature_to_gene_sets[feature] = gene_sets_matched

        feature_to_n_gene_set[feature] = len(gene_sets_matched)

feature_to_n_gene_set = pd.Series(feature_to_n_gene_set).sort_values()

print(feature_to_n_gene_set)

feature_x_cell_line = feature_x_cell_line.loc[feature_to_n_gene_set.index, :]

feature_x_cell_line.shape

In [ ]:
scores_directory_path = "{}/{}/{}_scores/".format(
    output_directory_path, feature_name, sea_method
)

kraft.path.path(scores_directory_path)

for feature, se in feature_x_cell_line.iterrows():

    directory_path = "{}/{}/".format(scores_directory_path, feature)

    if not os.path.isdir(directory_path):

        kraft.path.path(directory_path)

        kraft.function_heat_map.make(
            se,
            gene_set_x_cell_line,
            kraft.information.get_ic,
            n_job=8,
            n_sampling=0,
            n_permutation=0,
            plot=False,
            directory_path=directory_path,
        )

In [ ]:
feature_x_cell_line = feature_x_cell_line.loc[
    100 <= (~feature_x_cell_line.isna()).sum(axis=1), :
]

# feature_x_cell_line

In [ ]:
feature_to_scores = {}

feature_to_gene_sets = {}

feature_to_statistic = {}

for feature in feature_x_cell_line.index.to_numpy():

    scores = pd.read_table(
        "{}/{}/{}_scores/{}/scores.tsv".format(
            output_directory_path, feature_name, sea_method, feature
        ),
        sep="\t",
        index_col=0,
    )

    feature_to_scores[feature] = scores

    scores = scores.loc[(1 / 3) < scores.loc[:, "Score"], :]

    if 0 < scores.shape[0]:

        scores = scores.sort_values("Score").iloc[-10:, :]

        feature_to_gene_sets[feature] = scores.index.to_numpy()

        feature_to_statistic[feature] = np.median(scores.loc[:, "Score"].to_numpy())

feature_statistics = pd.Series(feature_to_statistic, name="Statistic").sort_values()

feature_statistics.index.name = "Feature"

In [ ]:
# TODO: plot n_gene_set together
kraft.plot.plot_x_y(
    feature_statistics.index.to_numpy(),
    feature_statistics.to_numpy(),
    feature_statistics.index.name,
    feature_statistics.name,
)

In [ ]:
for feature in feature_statistics.index.to_numpy()[-3:]:

    kraft.function_heat_map.summarize(
        feature_x_cell_line.loc[feature, :],
        {
            "Gene Set": {
                "df": gene_set_x_cell_line.loc[feature_to_gene_sets[feature], :],
                "data_type": "continuous",
                "emphasis": "+",
            }
        },
        {"Gene Set": feature_to_scores[feature]},
        se_data_type="continuous",
    )